## Toy problem for DQN, before applying DQN to mag inversion

#### Have square with length dx = 3, dy = 3 in subsurface, infinite depth, each measurement is vertical measurement down, return 1 if hit square, -1 if do not hit square. Goal is to determine position of square. Similar RL implementation to inversion problem since state is 2D image, reward is determined at the end (ground truthing), action is moving into one of the 4 consecutive cells, objective is getting as close to reality as possible, by determining optimal survey positions. The benefit of doing this toy problem first is that it is easy to see if the DQN will find the optimal solution

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import random


In [37]:
random.randint(0,1)

0

In [38]:
#setting up subsurface model:

M = np.zeros([10,10]) #generate grid of zeros

tlc = [random.randint(0,5),random.randint(0,5)] #top left corner

M[tlc[0]:tlc[0] + 5, tlc[1]:tlc[1] + 5] = 1
M

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [39]:
M[9,0]

0.0

In [40]:
not False

True

In [224]:
#setting up actions:

sp = [0,0] #starting position

a = random.randint(1,4) #1 right, 2 up, 3 left, 4 down

def movef(action, position):
    moved = False
    newposition = position #make the new position the current position, so position does not change if move is invalid
    if action == 1 and position[1] != 9: #right
        newposition = [position[0], position[1]+1]
        moved == True
    if action == 2 and position[0] != 0: #up
        newposition = [position[0]-1, position[1]]
        moved == True
    if action == 3 and position[1] != 0: #left
        newposition = [position[0], position[1]-1]
        moved == True
    if action == 4 and position[0] != 9: #down
        newposition = [position[0] + 1, position[1]]
        moved == True
    return newposition

In [42]:
M[7,6]

0.0

In [69]:
movef(4, [5,5])

[6, 5]

In [108]:
state = np.zeros([len(M), len(M)])
sp = [0,0] #start at top left corner

#0 denotes original "move", it does not move 
movelist = [0,4,4,4,4,1,2,2,1,4,4,1,1,1,1,1,4,3,3,4,4,4] 
#but surveys at original position

p = sp 



#go through moves
for move in movelist:
    p = movef(move, p)
    state[p[0], p[1]] = M[p[0], p[1]] * 2 - 1
    
#in RL, the next move will be determined from the policy and current state

In [109]:
state

array([[-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -1., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [110]:
M

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [316]:
#now for the reward, we do ground truthing, by fitting a 5x5 square to the
#state data, and then determining the misfit

def fitstate(S): #takes in the state
    fit = False

    while(not fit):
        #generate possible fit model:
        fM = np.zeros([10,10]) #fit Model
        ftlc = [random.randint(0,5),random.randint(0,5)] #fit top left corner
        fM[ftlc[0]:ftlc[0] + 5, ftlc[1]:ftlc[1] + 5] = 1

        fit = True #make fit true, but false if fit does not agree with survey
        for i in range(len(fM)):
            for j in range(len(fM[i])):
                if(S[i,j] != 0 and fM[i,j] * 2 - 1 != S[i,j]):
                    fit = False
    return fM

In [317]:
fM = fitstate(state)

In [318]:
M[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [319]:
state

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.],
       [-1.,  1.,  1.,  1.,  0.,  1., -1.,  0.,  0.,  0.],
       [-1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.],
       [-1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [320]:
M

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [321]:
fM

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [322]:
#now get reward, ~reciprocal of RMS

In [323]:
def RMSreward(real, fit):
    real = np.array(real)
    fit = np.array(fit)
    RMS = np.sqrt(np.average((real - fit)**2))
    reward = 1/(RMS + 1)
    return reward

In [324]:
real = [1,1,1,1]
fit = [1,1,1,2]
RMSreward(real, fit)

0.6666666666666666

In [325]:
np.sum(np.ones([10,10]))

100.0

In [326]:
RMSreward(fM, M) #perfect fit

1.0

### Want to start in the centre have fixed total amount of moves

In [429]:
sp = [5,5]

mL = [0]
for i in range(25): #20 random moves
    mL.append(random.randint(1,4))


In [430]:
p = sp 
state = np.zeros([len(M), len(M)])

#go through moves
for move in mL:
    p = movef(move, p)
    state[p[0], p[1]] = M[p[0], p[1]] * 2 - 1

In [431]:
state

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [432]:
RMSreward(fitstate(state), M) 

0.7597469266479578